# Frequency Calibration

This is a brief note about frequency calibration on the RTL-SDRs

## The dongle

Recall our signal path contains a "Local Oscillator" as one of the things feeding in to the "Mixer"

<img src="https://www.rtl-sdr.com/wp-content/uploads/2018/03/rtl_sdr_behavioural_model.png">

# Local Oscillator

How does the local oscillator work?

There are at least two components:

1. a "crystal oscillator" that produces a fixed-frequency tone at 28.8 MHz
1. a "phase-locked loop" that lets you tune to a specific frequency

## Crystal Oscillator

The crystal oscillator produces a fixed-frequency signal, but that signal can drift a little with temperature.



## Phase-Locked Loop

The PLL (phase-locked loop) is a device used to synchronize a periodic waveform with a reference periodic waveform (i.e., keep the phase the same over time, assuming the two waves have the same frequency).

<!--https://wirelesspi.com/phase-locked-loop-pll-in-a-software-defined-radio-sdr/-->
LOTS of detail here: https://wirelesspi.com/phase-locked-loop-pll-in-a-software-defined-radio-sdr/ but we won't go into it now

# LO (local oscillator): what do you need to know?

The Crystal Oscillator + Phase-Locked Loop constitute our tunable LO.

It is not perfectly accurate, it has a fractional drift of up to ~100 ppm (parts per million).

If we want to observe 1420 MHz, that means there could be $\frac{100}{1000000} 1,420,000,000 \mathrm{~Hz} = 142,000 \mathrm{~Hz}$ offset.

## Review: What's "100 ppm" in velocity?

If we have an offset of 100 parts per million, what is that in velocity?

In [1]:
from astropy import constants, units as u
(100/1e6 * constants.c).to(u.km/u.s)

<Quantity 29.9792458 km / s>

So that offset could be pretty big!

## Calibrating the offset

The offset is relatively steady with time - it probably doesn't change much once the SDR has reached its steady running temperature.  It also is pretty steady for each individual dongle.

We can therefore correct for this offset by measuring it against a known frequency.

## Calibrating the offset

What frequencies do we know clearly?

We want something that is *narrow-band*: it needs to have a signal small enough that we can measure the offset to ~1 part per million.

The NOAA (National Oceanic and Atmospheric Administration) broadcasts weather alerts on a set narrow-band frequency.  In Gainesville, that is at 162.475 MHz

## SDR-SHARP aside here

## Calibrating the offset

The code [scan_calibrator_signals.py](https://github.com/keflavich/RTL_SDR_HI_Observations/blob/master/scan_calibrator_signals.py) records a signal at close to 162.475, then measures where, i.e. at what frequency, the signal is the brightest.

It then does the conversion to ppm and reports the "offset" you need to apply to get to the correct signal.